In [28]:
import pandas as pd
from IPython.display import display, Markdown
import numpy as np

In [45]:
dataframe = pd.read_csv("csvs/glicose.csv")
print(dataframe)

   inicial   final
0      105      99
1       96     101
2      107      98
3      102     102
4       99      88
5       88      98
6      100      90
7      105      98
8       89      97
9      123     112


In [46]:
# NUMERO DE TRATAMENTOS E NUMERO DE REPETIÇÕES
r, k = dataframe.shape

## GRAUS DE LIBERDADE = GL = DF ## 
# Numero de Variaveis Independentes
# gl_trat = k - 1
df_between = k - 1
# gl_erro
df_within = (r*k) - k
print(f"(dfb) DF tratamentos(entre colunas): {round(df_between, 2)}")
print(f"(dfw) DF erros(dentro das colunas): {round(df_within, 2)}")

(dfb) DF tratamentos(entre colunas): 1
(dfw) DF erros(dentro das colunas): 18


In [48]:
# SUM OF SQUARE - SSQ
# Soma dos quadrados - SQ

mean_of_all_data = dataframe.to_numpy().mean()

# SSB ou SQTRAT
sum_of_square_between = 0 
# sum_of_square_between = sum([*map(lambda n: ((n - mean_of_all_data) ** 2) * r, dataframe.mean())])

for group in dataframe.items():
    column_mean = group[1].mean()
    for value in group[1]:
        sum_of_square_between += (column_mean - mean_of_all_data) ** 2  

print(f"(SSB = SQTrat) Soma de quadrados entre colunas: {sum_of_square_between}")

# SSW ou SQerro
sum_of_square_within = 0
# sum_of_square_within = sum([*map(lambda n: sum([*map(lambda m: (m - dataframe[n].mean())**2, dataframe[n])]), dataframe.columns)])

for value in dataframe.items():
    column_mean = value[1].mean()
    sum_of_column = sum([*map(lambda n: (n - column_mean) ** 2, value[1])])
    sum_of_square_within += sum_of_column

print(f"(SSW = SQErro) Soma de quadrados dentro de colunas: {sum_of_square_within}")

# SST ou SQTotal
sum_of_square_total = 0

for group in dataframe.iterrows():
    for value in group[1]:
        sum_of_square_total += (float(value) - mean_of_all_data) ** 2
print(f"(SST = SQTotal) Soma de quadrados Totais: {sum_of_square_total}")

(SSB = SQTrat) Soma de quadrados entre colunas: 48.05000000000027
(SSW = SQErro) Soma de quadrados dentro de colunas: 1280.5
(SST = SQTotal) Soma de quadrados Totais: 1328.55


In [32]:
# MSw ou QMerro 
mean_square_within = sum_of_square_within / df_within
print(f"(MSw = QMerro) Media de quadrados entre as colunas {round(mean_square_within, 2)}")
# MSb ou QMtrat
mean_square_between = sum_of_square_between / df_between
print(f"(MSb = QMtrat) Media de quadrados entre colunas {round(mean_square_between, 2)}")
    

(MSw = QMerro) Media de quadrados entre as colunas 71.14
(MSb = QMtrat) Media de quadrados entre colunas 48.05


In [33]:
calculated_fischer = mean_square_between / mean_square_within
print(f"Fischer estatisticas: {calculated_fischer}")

variance_coefficient = np.sqrt(mean_square_within) / mean_of_all_data
variance_coefficient = variance_coefficient * 100
print(f"CV : {variance_coefficient} %")

Fischer estatisticas: 0.6754392815306556
CV : 8.44705787272909 %


In [34]:
tabela_dict = {
"Fonte de Váriação" : ["Tratamento", "Erro", "Total", "Média Geral", "CV(%)"],
"GL": [df_between.__round__(2), df_within.__round__(2), df_between + df_within, "", ""],
"SQ": [sum_of_square_between.__round__(2), sum_of_square_within.__round__(2), sum_of_square_total.__round__(2),"", ""], 
"QM": [mean_square_between.__round__(2),mean_square_within.__round__(2),"","", ""], 
"F" : [calculated_fischer.__round__(2),"","", mean_of_all_data.__round__(2),variance_coefficient.__round__(2)]}

dataframe_tabela = pd.DataFrame(tabela_dict)

display(Markdown(dataframe_tabela.to_markdown(index=False)))

| Fonte de Váriação   | GL   | SQ      | QM    | F     |
|:--------------------|:-----|:--------|:------|:------|
| Tratamento          | 1    | 48.05   | 48.05 | 0.68  |
| Erro                | 18   | 1280.5  | 71.14 |       |
| Total               | 19   | 1328.55 |       |       |
| Média Geral         |      |         |       | 99.85 |
| CV(%)               |      |         |       | 8.45  |

In [35]:
dataframe = pd.read_csv("csvs/tabela_fischer.csv")

ftab = list(dataframe.iterrows())[df_within -1][1][f"{df_between}"]
print(f"Valor na Tabela de Fischer: {ftab}")
print(f"Valor calculado na tabela de variação: {calculated_fischer}")
if ftab < calculated_fischer:
    print("Existe Diferença Significativa")
else: 
    print("Não Existe Diferença Significativa")

Valor na Tabela de Fischer: 4.414
Valor calculado na tabela de variação: 0.6754392815306556
Não Existe Diferença Significativa
